# 测试集数据生成

## 利用ragas生成测试集

### 以下是官方案例
1. 使用langchain读取文档
    ```python

    from langchain_community.document_loaders import DirectoryLoader

    path = "Sample_Docs_Markdown/"
    loader = DirectoryLoader(path, glob="**/*.md")
    docs = loader.load()
    ```

2. 将选定的大模型封装在ragas的``LangchainLLMWrapper``与``LangchainEmbeddingsWrapper``中
    ```python
    from langchain_openai import AzureChatOpenAI
    from langchain_openai import AzureOpenAIEmbeddings
    from ragas.llms import LangchainLLMWrapper
    from ragas.embeddings import LangchainEmbeddingsWrapper
    generator_llm = LangchainLLMWrapper(AzureChatOpenAI(
        openai_api_version="2023-05-15",
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    ))

    # init the embeddings for answer_relevancy, answer_correctness and answer_similarity
    generator_embeddings = LangchainEmbeddingsWrapper(AzureOpenAIEmbeddings(
        openai_api_version="2023-05-15",
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    ))
    ```

3. 生成测试数据集
    ```python
    from ragas.testset import TestsetGenerator

    generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
    dataset = generator.generate_with_langchain_docs(docs, testset_size=10)
    ```

4. 最后将测试集转化为pandas的DataFrame即可
    ```python
    dataset.to_pandas()
    ```

### 以下是实际操作

In [2]:
# 引用ollama的大模型
# from langchain_ollama import ChatOllama
# from langchain_ollama import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings

# ragas的Wrapper
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# 测试集生成器
from ragas.testset import TestsetGenerator

# 文档读取器
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# 大模型配置类
from ragas.run_config import RunConfig

# 配置生成的问题类型
from ragas.testset.synthesizers import SingleHopSpecificQuerySynthesizer, MultiHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer

# 处理文本大小
from langchain_core.documents import Document
from langchain.text_splitter import TokenTextSplitter, RecursiveCharacterTextSplitter


/home/lct/miniconda3/envs/evaluate/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 读取指定目录下的txt文件
# path = "/home/lct/Ragas/测试集参考文档"
# loader = DirectoryLoader(path=path, glob='*.txt')

path = "../测试集参考文档/工业流程Turned.txt"
loader = TextLoader(path, encoding='utf-8')
base_docs = loader.load()

In [4]:
base_docs

[Document(metadata={'source': '/home/lct/Ragas/测试集参考文档/工业流程Turned.txt'}, page_content='== 制茶 ==\n制茶是制茶程序\n-- 制茶的介绍 --\n萎凋：使叶质柔软，可塑性大，便于造形。\n作青：摇青与堆置交替进行，由摇青与晾青两个过程所组成。作青室温25℃左右为宜，相对湿度80%左右。\n摇青：将萎凋的茶叶放在竹筛内，来回揉，使叶片边缘经过摩擦，叶缘细胞受损，经摊置失水，叶中多酚类在酶的作用下，渐渐被氧化，形成茶叶特有的品质。摇青先轻后重，以免梗叶折断造成死青。这样会破坏茶叶的质量。\n铁观音宜重摇，延长作青时间；摇青5～6次，使叶缘成朱砂红色，叶中呈黄绿色（半熟香蕉皮）。 摇青过程中，组织因振动而增加细胞吸水力，茎梗的水分往叶脉、叶片输送，香味及可溶性物质输送到叶片，水分由叶片蒸发，摇青后叶子恢复舒张状态，称为「还青」。还青后静置晾青，水分继续蒸发，又呈萎凋状态，经过5～7次摇青、晾青交替进行，叶片呈绿叶红边，状如汤匙状，外观硬挺，手感软，散发出浓郁桂花香，即为作青的适度。\n北京时间2022年11月29日，中国申报的“中国传统制茶技艺及其相关习俗”，在摩洛哥拉巴特召开的联合国教科文组织保护非物质文化遗产政府间委员会第17届常会上通过评审，列入联合国教科文组织人类非物质文化遗产代表作名录。[1]\n- 制茶的中文名:制茶\n- 制茶的萎    凋:柔软.可塑性大.便于造形\n- 制茶的产    品:乌龙茶、铁观音\n- 制茶的产    地:安溪、长坑\n-- 制茶的制茶工艺 --\n晾青：时间先短后长，摊青先薄后厚。杀青：是利用高温处理新鲜的茶叶，使之变软，保持绿色，并失去一部分水分，便于揉捻。温度：280～300℃。目的：利用高温杀死青叶中的催化酶，使酶失去活性，保持叶片绿色，借热化学反应，消除叶中青臭、苦、涩味，转化为具有花香醇味的杀青叶，能减少茶叶受损情况。揉捻：茶叶含水量60%为宜，适于整形；提高叶温，揉捻可塑性增强。闷堆：将锅炒的茶叶，堆积盖布闷黄。发酵：决定茶叶色、香、味品质的关键。发酵由揉捻开始，因揉捻的压力作用，叶细胞受损，多酚类霉促氧化受损，聚合加速进行，而开始发酵。发酵温度初期32℃～40℃。后期15℃～18℃。发酵初期氧化速度大，要充分供氧，室内宜通风

In [5]:
docs = []

for base_doc in base_docs:
    source = base_doc.metadata

    splitter = TokenTextSplitter(chunk_size=600)
    chunks = splitter.split_text(base_doc.page_content)

    for chunk in chunks:
        docs.append(Document(metadata=source, page_content=chunk))


In [6]:
# 包装大模型
# llm = ChatOllama(model="qwen2.5:32b", num_ctx=32768, base_url='http://localhost:11434')
# emb = OllamaEmbeddings(model="bge-m3", base_url='http://localhost:11434')

llm = ChatOllama(model="qwen2.5:32b", num_ctx=32768, base_url='http://localhost:11434')
emb = OllamaEmbeddings(model="bge-m3", base_url='http://localhost:11434')


ollama_chat = LangchainLLMWrapper(langchain_llm=llm, run_config=RunConfig(timeout=1200))
ollama_emb = LangchainEmbeddingsWrapper(embeddings=emb)

/tmp/ipykernel_594/1168921987.py:5: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="qwen2.5:32b", num_ctx=32768, base_url='http://localhost:11434')
/tmp/ipykernel_594/1168921987.py:6: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  emb = OllamaEmbeddings(model="bge-m3", base_url='http://localhost:11434')


In [6]:
# 配置测试集的问题分布（未启用）
query_distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=ollama_chat), 0.4),
    (MultiHopAbstractQuerySynthesizer(llm=ollama_chat), 0.3),
    (MultiHopSpecificQuerySynthesizer(llm=ollama_chat), 0.3)
]

In [7]:
# 生成测试数据
generator = TestsetGenerator(llm=ollama_chat, embedding_model=ollama_emb)
dataset = generator.generate_with_langchain_docs(testset_size=20, documents=docs)

Generating Samples: 100%|██████████| 20/20 [01:44<00:00,  5.22s/it]


In [9]:
df = dataset.to_pandas()
df

,user_input,reference_contexts,reference,synthesizer_name
0,什么是摩洛哥拉巴特在中国传统制茶技艺申报中的作用？,[样会破坏茶叶的质量。\n铁观音宜重摇，延长作青时间；摇青5～6次，使叶缘成朱砂红色，叶中呈...,摩洛哥拉巴特是联合国教科文组织会议的举办地，在这里审议了中国申报的“中国传统制茶技艺及其相关...,single_hop_specifc_query_synthesizer
1,What happened at the 17th session of the Inter...,[片呈绿叶红边，状如汤匙状，外观硬挺，手感软，散发出浓郁桂花香，即为作青的适度。\n北京时间...,The Chinese application for 'Traditional Tea-m...,single_hop_specifc_query_synthesizer
2,What is the main tea production area mentioned...,[�龙茶、铁观音\n- 制茶的产 地:安溪、长坑\n-- 制茶的制茶工艺 --\n晾青...,The main tea production area mentioned in the ...,single_hop_specifc_query_synthesizer
3,Whaet is the prcess of wochui in Pu'er tea pro...,[��捻已算告一段落，剩下的只是干燥，但「后发酵茶」在杀青、揉捻后有一堆放的过程称为「渥堆」...,The process of渥堆 (wochui) involves stacking th...,single_hop_specifc_query_synthesizer
4,什么是茶的圆饼状包装及其优点？,[之四，利于贮藏运销，通常为了能使内外干燥一致，常采用二次干燥法，先使其达到七、八成干燥，然...,圆饼状是紧压茶的一种形状。紧压茶通过将制成的茶叶蒸软后加压成块状来制作，这不仅便于运输和贮藏...,single_hop_specifc_query_synthesizer
5,红茶的发酵过程是如何影响其颜色和味道的？,[干燥＞红茶发酵：发酵是制造红茶的关键，又称『渥红』，因叶片中含有生物催化剂『多酚氧化酶』，...,红茶的发酵是制造红茶的关键步骤，也称为『渥红』。在这个过程中，茶叶中的多酚氧化酶在不经过高温...,single_hop_specifc_query_synthesizer
6,什么是红碎茶？,[�的。分类：1.小种红茶：红汤红叶，有松烟香，为似桂圆汤。熏焙：茶叶薄摊于竹筛中，地上堆松...,红碎茶是揉捻时用机器将叶片切碎，成颗粒形碎片。,single_hop_specifc_query_synthesizer
7,什么是银针茶?,[＞杀青＞初揉＞初烘＞包揉＞复烘＞干燥＞青茶（乌龙茶）属半发酵茶，介于红茶与绿茶之间的茶类。...,银针又称白毫银针，是采摘细嫩茶叶加工时不炒不揉，使白茸毛在茶外表完整保存下来的白茶品种。,single_hop_specifc_query_synthesizer
8,What is the origin of '君山银针'?,[�眉：采摘一芽二三叶。4.寿眉：制造时先经抽针，抽摘出之茶芽做银针，其它叶片制贡眉，制造时...,君山银针是黄芽茶的一种，其原料细嫩，主要采摘单芽或一芽一叶加工而成，产自湖南岳阳洞庭湖的君山。,single_hop_specifc_query_synthesizer
9,什么是黑茶？,[“霍山黄芽”。\n~~ 黑茶 ~~\n鲜叶＞杀青＞初揉＞渥堆＞复揉＞干燥＞黑茶黑茶由于原料...,黑茶由于原料粗老，并且在制造过程中有一段较长的堆积发酵时间，因此叶色多呈暗褐色。,single_hop_specifc_query_synthesizer


In [ ]:
df.to_json('/home/lct/Ragas/测试数据.json')

In [2]:
import pandas as pd

df = pd.read_csv('/home/lct/Ragas/测试数据.csv')

In [4]:
df.iloc[17]['user_input']

'如何用冻顶熏桂花，并且通过不同的焙火程度来影响茶叶的颜色和口感？'

In [7]:
df.iloc[17]['reference_contexts']

"['<1-hop>\\n\\n�一批新鲜的花朵，再重复制作一次，这样制成的茶就称为双熏花茶，如果还嫌不够，还可以再重复熏制一次，那就是三熏花茶了。但大家得记住，我们是在喝茶，只是藉花衬托茶味而已，所以制茶老师父会提醒我们：「七分茶，三分花」。什么茶配什么花有没有一定准则？没有，但一般人会考虑相不相配的问题，如茉莉花与桂花，比较起来茉莉花较年轻，桂花较成熟，所以我们会用清茶或绿茶熏茉莉花，用冻顶或铁观音熏桂花。焙火如果我们想让制成的茶有股火香，感觉得比较温暖一点，可拿来用火烘焙。焙火轻重也会造成不同的风味，焙火轻者喝来感觉比较生，焙火重者喝来感觉比较熟。我们从�', '<2-hop>\\n\\n�用冻顶或铁观音熏桂花。焙火如果我们想让制成的茶有股火香，感觉得比较温暖一点，可拿来用火烘焙。焙火轻重也会造成不同的风味，焙火轻者喝来感觉比较生，焙火重者喝来感觉比较熟。我们从外观上如何看出焙火的轻重？焙火轻者，颜色较亮，焙火重者，颜色较暗，这颜色包括茶干的颜色与冲泡后茶汤的颜色。在发酵时我们谈到过：发酵愈轻，颜色愈绿，发酵愈重，颜色愈红。焙火所影响的是颜色的深浅，也就是明度的高低，焙火愈重，明度愈低，焙火愈轻，明度愈高。在品饮的口感上有何差异呢？喝轻焙火的茶有如吃清蒸清炒的菜，喝重焙火的茶有如吃红烧的。对身体的效应有何不同呢？�']"

In [5]:
df.iloc[17]['reference']

'使用冻顶或铁观音熏桂花，可以制作出带有花香的茶。焙火轻者喝起来感觉比较生，颜色较亮；焙火重者喝起来感觉比较熟，颜色较暗。轻焙火的茶有如吃清蒸清炒的菜，而重焙火的茶则像红烧的味道。'